<a href="https://colab.research.google.com/github/HromovAndrey/II_mood_3_part_4/blob/Practika/%D0%9F%D1%80%D0%B0%D0%BA%D1%82%D0%B8%D0%BA%D0%B0_lightgbm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Імпорт модулів

In [76]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import numpy as np

Дані про продаж будинків

In [77]:
df = pd.read_csv("https://raw.githubusercontent.com/HalyshAnton/IT-Step-Pyton-AI/main/module3/data/AmesHousing.csv",
                 index_col=0)

# Завдання 1
Виведіть основіні числові характеристики, кількість пропущенних значень для кожного стовпчика

In [78]:
df.describe()
missing_value = df.isnull().sum()
missing_value

PID                 0
MS SubClass         0
MS Zoning           0
Lot Frontage      490
Lot Area            0
                 ... 
Mo Sold             0
Yr Sold             0
Sale Type           0
Sale Condition      0
SalePrice           0
Length: 81, dtype: int64

# Завдання 2
Розділіть дані на тестові та тренувальні(цільова ознака `SalePrice`)

In [79]:
y = df['SalePrice']
X = df.drop(columns='SalePrice')

In [80]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2,
                                                    random_state=42)


# Завдання 3
Створіть препроцесор для заповнення пропусків та натренуйте його

In [81]:
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object']).columns

In [82]:
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median'))
])

In [83]:
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

In [84]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
        ])

In [85]:
X_train_preprocessed = preprocessor.fit_transform(X_train)
X_test_preprocessed = preprocessor.transform(X_test)

# Завдання 4
Створіть модель та натренуйте її

In [86]:
model = LinearRegression()
model.fit(X_train_preprocessed, y_train)

LinearRegression()

# Завдання 5
Виведіть метрики, важливість онак та графік метрик на тренувальних та тестових даних

In [87]:
import matplotlib.pyplot as plt
import numpy as np

# Прогнозування на тренувальних та тестових даних
y_train_pred = model.predict(X_train_preprocessed)
y_test_pred = model.predict(X_test_preprocessed)

# Функція для обчислення метрик
def get_metrics(y_true, y_pred):
    return {
        "MSE": mean_squared_error(y_true, y_pred),
        "R2": r2_score(y_true, y_pred)
    }

# Обчислення метрик
train_metrics = get_metrics(y_train, y_train_pred)
test_metrics = get_metrics(y_test, y_test_pred)

# Важливість ознак
feature_importance = np.abs(model.coef_)
feature_names = numeric_features.tolist() + preprocessor.named_transformers_['cat']['onehot'].get_feature_names_out(categorical_features).tolist()

# Виведення метрик
print("Train metrics:", train_metrics)
print("Test metrics:", test_metrics)

# Виведення важливості ознак
importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importance})
importance_df = importance_df.sort_values(by='Importance', ascending=False)

print(importance_df.head(10))

# Графік важливості ознак
plt.figure(figsize=(10, 8))
plt.barh(importance_df['Feature'], importance_df['Importance'])
plt.xlabel("Feature Importance")
plt.title("Feature Importance for House Price Prediction")
plt.gca().invert_yaxis()
plt.show()

# Графік метрик на тренувальних та тестових даних
metrics_df = pd.DataFrame({'Train': train_metrics.values(), 'Test': test_metrics.values()}, index=train_metrics.keys())
metrics_df.plot(kind='bar')
plt.title("Metrics on Train and Test Data")
plt.ylabel("Metric Value")
plt.show()


NameError: name 'mean_squared_error' is not defined

In [ ]:
y_train_pred = model.predict(X_train_preprocessed)
y_test_pred = model.predict(X_test_preprocessed)

In [ ]:
def get_metrics(y_true, y_pred):
    return {
        "MSE": mean_squared_error(y_true, y_pred),
        "R2": r2_score(y_true, y_pred)
    }

In [ ]:
train_metrics = get_metrics(y_train, y_train_pred)
test_metrics = get_metrics(y_test, y_test_pred)

# Завдання 6
Об'єднайте препроцесор та натреновану модель і збережіть її